In [ ]:
import requests
import glob
import os
import sys
from PIL import Image
import numpy as np
import plotly.express as px
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import pandas as pd
import urllib.request

In [ ]:
# Adding information about user agent
opener=urllib.request.build_opener()
opener.addheaders=[('User-Agent','Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1941.0 Safari/537.36')]
urllib.request.install_opener(opener)

label_encoder = {"not_ramen": 0, "ramen":1}
try:
  [os.makedirs(k) for k in label_encoder.keys()]
except:
  pass

In [ ]:
ramen_img_urls = [
  "https://www.forkknifeswoon.com/wp-content/uploads/2014/10/simple-homemade-chicken-ramen-fork-knife-swoon-01.jpg",
  "https://upload.wikimedia.org/wikipedia/commons/thumb/e/ec/Shoyu_ramen%2C_at_Kasukabe_Station_%282014.05.05%29_1.jpg/1200px-Shoyu_ramen%2C_at_Kasukabe_Station_%282014.05.05%29_1.jpg",
  "https://hips.hearstapps.com/hmg-prod/images/190208-delish-ramen-horizontal-093-1550096715.jpg",
  "https://www.elmundoeats.com/wp-content/uploads/2021/02/FP-Quick-30-minutes-chicken-ramen.jpg",
  "https://assets.bonappetit.com/photos/5e3c7a3c866b940008106763/5:7/w_2485,h_3479,c_limit/HLY-Veggie-Ramen-16x9.jpg",
  "https://www.halfbakedharvest.com/wp-content/uploads/2021/10/Easy-Ginger-Chicken-and-Spinach-Ramen-1.jpg",
  "https://images.media-allrecipes.com/userphotos/9020363.jpg",
  "https://glebekitchen.com/wp-content/uploads/2017/04/tonkotsuramenfront.jpg",
  "https://www.modernfarmhouseeats.com/wp-content/uploads/2021/03/chili-lime-shrimp-ramen-2-scaled.jpg",
  "https://peasandcrayons.com/wp-content/uploads/2020/04/spicy-vegetarian-ramen-recipe-3.jpg",
  "https://steamykitchen.com/wp-content/uploads/2011/04/miso-ramen-recipe-feature-20912.jpg",
  "https://content.hy-vee.com/remote.axd/3f4c2184e060ce99111b-f8c0985c8cb63a71df5cb7fd729edcab.ssl.cf2.rackcdn.com/media/15684/ramenbowls.jpg?v=1&mode=crop&width=800&height=640&upscale=false",
  "https://www.theflavorbender.com/wp-content/uploads/2019/01/Easy-Chicken-Ramen-Featured-500x500.jpg",
  "https://thewoodenskillet.com/wp-content/uploads/2021/04/pork-belly-ramen-recipe-8.jpg",
  "https://assets.tmecosys.com/image/upload/t_web767x639/img/recipe/vimdb/210551.jpg",
  "https://www.kitchensanctuary.com/wp-content/uploads/2020/12/Quick-Chicken-Ramen-square-FS-22.jpg",
  "https://thesaltymarshmallow.com/wp-content/uploads/2018/11/ramen-noodles-featured-500x375.jpg",
  "https://www.seriouseats.com/thmb/kYc-sDcKzfnppXnqyZnbuJcmxDY=/1500x844/smart/filters:no_upscale()/__opt__aboutcom__coeus__resources__content_migration__serious_eats__seriouseats.com__recipes__images__2015__02__20150215-vegan-ramen-recipe-01-3fbc618f3cc845be81188529cf5bc2c3.jpg",
  "https://sixhungryfeet.com/wp-content/uploads/2021/08/Miso-Tahini-Ramen-7.jpg",
  "https://assets.epicurious.com/photos/54ad70036529d92b2c046e6b/5:4/w_885,h_708,c_limit/51187270_shoyu-ramen_1x1.jpg",
  "https://www.killingthyme.net/wp-content/uploads/2015/10/homemade-ramen-bowls-5.jpg",
  "https://imagesvc.meredithcorp.io/v3/mm/image?url=https%3A%2F%2Fstatic.onecms.io%2Fwp-content%2Fuploads%2Fsites%2F9%2F2013%2F12%2F06%2F200905-xl-shoyu-ramen.jpg&q=60",
  "https://www.budgetbytes.com/wp-content/uploads/2021/01/Spicy-Peanut-Butter-Ramen-fork.jpg",
  "https://www.halfbakedharvest.com/wp-content/uploads/2021/01/30-Minute-Spicy-Miso-Chicken-Katsu-Ramen-1.jpg",
  "https://images.themodernproper.com/billowy-turkey/production/posts/2018/easy-chicken-ramen-soup-9.jpg?w=1200&auto=compress%2Cformat&fit=crop&dm=1599771072&s=882ef30b867024847ad03db0edad3c5f",
  "https://thefoodcharlatan.com/wp-content/uploads/2021/10/Easy-Pork-Ramen-Recipe-Slow-Cooker-8.jpg",
  "https://www.justonecookbook.com/wp-content/uploads/2020/04/Spicy-Shoyu-Ramen-2167-I.jpg",
  "https://www.thespruceeats.com/thmb/Zo2LZw1qbeIwaxD1bak6iikE_As=/1708x1708/smart/filters:no_upscale()/mariekeyesramen-56e593233df78c5ba0573041.jpg",
  "https://i1.wp.com/seonkyounglongest.com/wp-content/uploads/2020/09/Easy-Tonkotsu-26-mini.jpg?fit=1000%2C667&ssl=1",
  "https://images.eatsmarter.com/sites/default/files/styles/1600x1200/public/chicken-ramen-with-vegetables-676167.jpg",
  "https://www.modernfarmhouseeats.com/wp-content/uploads/2021/03/chili-lime-shrimp-ramen-2-500x500.jpg",
  "https://images.aws.nestle.recipes/original/fece95affbbdc1138d4920a47209a7de_ramen_920.jpg",
  "https://images.immediate.co.uk/production/volatile/sites/30/2021/05/Roast-broccoli-and-sesame-ramen-noodle-bowls-f57730e.jpg",
  "https://cdn.vox-cdn.com/thumbor/AYwWLPgIhB2PKlmsjIgfEaiQYUM=/0x0:2400x1800/1200x900/filters:focal(1008x708:1392x1092)/cdn.vox-cdn.com/uploads/chorus_image/image/58040171/Rondo_ramen_20220511_114339.46.jpg",
  "https://inquiringchef.com/wp-content/uploads/2016/10/20-Minute-Chicken-Ramen-Stirfry-500x500.jpg",
  "https://www.kitchensanctuary.com/wp-content/uploads/2016/07/Pork-Ramen-square-FS-.jpg",
  "https://imagesvc.meredithcorp.io/v3/mm/image?url=https%3A%2F%2Fstatic.onecms.io%2Fwp-content%2Fuploads%2Fsites%2F9%2F2015%2F08%2F28%2F201405-xl-easy-chicken-ramen-soup-2000.jpg&q=60",
  "https://www.takeaway.com/foodwiki/uploads/sites/11/2019/08/ramen_2-1080x1080.jpg",
  "https://imgmedia.lbb.in/media/2021/11/619c62ce1134120ab583ce0d_1637638862209.jpg",
  "https://post.healthline.com/wp-content/uploads/2020/09/ramen-noodles-1200x628-facebook-1200x628.jpg",
  "https://images.getbento.com/accounts/8b8f5494ef5db25886edf0a771e62985/media/images/72916Ani_Ramen_9.jpg?w=1800&fit=max&auto=compress,format&h=1800",
  "https://cdn.vox-cdn.com/thumbor/cMM-Q5UB1ZQColUv6wTmw36hjqQ=/0x0:2048x1365/1200x900/filters:focal(861x519:1187x845):no_upscale()/cdn.vox-cdn.com/uploads/chorus_image/image/62543859/iza_ramen.0.0.0.jpg",
  "https://i.ytimg.com/vi/TGHbePXG7Oo/maxresdefault.jpg",
  "https://thefoodietakesflight.com/wp-content/uploads/2020/02/saucy-ramen-noodles-for-WP.png",
  "https://www.halfbakedharvest.com/wp-content/uploads/2020/07/15-Minute-Garlic-Butter-Ramen-Noodles-4-700x1050.jpg",
  "https://static.wixstatic.com/media/05788c_e122b9a23ab84ba0940a61dcae0ac784~mv2_d_1863_1242_s_2.jpg/v1/fill/w_462,h_452,al_c,q_80,usm_0.66_1.00_0.01,enc_auto/05788c_e122b9a23ab84ba0940a61dcae0ac784~mv2_d_1863_1242_s_2.jpg",
  "https://www.halfbakedharvest.com/wp-content/uploads/2020/07/15-Minute-Garlic-Butter-Ramen-Noodles-6-700x1050.jpg",
  "https://www.acouplecooks.com/wp-content/uploads/2020/02/Tofu-Ramen-017.jpg",
  "https://www.halfbakedharvest.com/wp-content/uploads/2020/07/15-Minute-Garlic-Butter-Ramen-Noodles-7-700x467.jpg",
  "https://www.seriouseats.com/thmb/1xvdMa4udyCKzGsOL2Tdfu99nLM=/610x458/filters:no_upscale():max_bytes(150000):strip_icc()/__opt__aboutcom__coeus__resources__content_migration__serious_eats__seriouseats.com__images__20110317-ramen-hacks-04-d062e50fa1e54b52a87f79b1a345d632.jpg",
  "https://www.plainchicken.com/wp-content/uploads/2020/01/spicy-steak-ramen-noodles-2-500x375.jpg",
  "https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/190208-delish-ramen-horizontal-093-1550096715.jpg"
  "https://www.ramenninjas.com/wp-content/uploads/2020/07/pork1.jpg",
  "https://preview.redd.it/jb9e2fv8rsv11.jpg?auto=webp&s=6a1a45e06a850e92a33a58484ae6e1084fadec9e"

]

for ramen_url in ramen_img_urls:
    try:
        urllib.request.urlretrieve(ramen_url, os.path.join(os.getcwd(),f"ramen/ramen_{ramen_img_urls.index(ramen_url)}.jpg"))
    except:
        print(f"{ramen_url} at {ramen_img_urls.index(ramen_url)} not working, trying next...")
    continue

In [ ]:
images_array = []
y_train = []

def show_image(img_array):
  fig = px.imshow(img_array)
  fig.show()

def img_to_array(folder, class_label, img_size, show_img = False):
  for img in glob.glob(folder):
    # if img.endswith("jpg"):
    try:
      loaded_img = Image.open(img)
      rz_img = Image.Image.resize(loaded_img, [img_size,img_size])
      img_array = np.asarray(rz_img).astype('float32').reshape(240,240,3)
      images_array.append(img_array)
      y_train.append(label_encoder[class_label])
      if show_img:
        show_image(img_array)
    except:
      print(f"{img} failed to be processed, trying the next...")
      continue
  



img_to_array(os.path.join(os.getcwd(),"ramen/*"), "ramen", 240)
x_train = np.asarray(images_array).astype('float32')
x_train = (x_train - 127.5) / 127.5

In [ ]:
from keras.models import Sequential, Model
import keras.layers as klayer

ramen_generator = Sequential(
    [klayer.Dense(128 * 60 * 60, input_dim=100, activation=klayer.LeakyReLU(0.2)),
    klayer.BatchNormalization(),
    klayer.Reshape((60,60,128)),
    klayer.UpSampling2D(),
    klayer.Conv2D(120, (5,5), padding= 'same', activation=klayer.LeakyReLU(0.2)),
    klayer.BatchNormalization(),
    klayer.Conv2D(60, (3,3), padding = 'same', activation=klayer.LeakyReLU(0.2)),
    klayer.BatchNormalization(),
    klayer.UpSampling2D(),
    klayer.Conv2D(3, (5,5), padding = 'same', activation='tanh')]
)

ramen_generator.summary()

In [ ]:
ramen_discriminator = Sequential([
    klayer.Conv2D(60, (5,5), padding= 'same',input_shape=(240,240,3), activation=klayer.LeakyReLU(0.2)),
    klayer.Dropout(0.4),
    klayer.Conv2D(60, (5,5), padding= 'same', activation=klayer.ActivityRegularization(0.2)),
    klayer.Dropout(0.5),
    klayer.Flatten(),
    klayer.Dense(1,activation='sigmoid')
])

ramen_discriminator.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam

ramen_discriminator.trainable = True
ramen_discriminator.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0001,beta_1=0.5))

In [ ]:
ramen_generator.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001,beta_1=0.5))
ramen_gan_input = klayer.Input(shape=(100,))
gen_img = ramen_generator(ramen_gan_input)
ramen_discriminator.trainable = False
ramen_gan_output = ramen_discriminator(gen_img)
ramenGAN = Model(inputs=ramen_gan_input, outputs=ramen_gan_output)
ramenGAN.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0001,beta_1=0.5))

In [ ]:
try:
  os.makedirs("gan_output")
except:
  pass

def train(num_ep, batch_size):
  batch_count = x_train.shape[0]
  print(f"batch count: {batch_count}")

  for epoch in range(num_ep):
    print(f"Epoch: {epoch}")
    for batch in range(batch_count):
      noise_input = np.random.rand(batch_size, 100)
      fake_imgs = ramen_generator.predict(noise_input, batch_size=batch_size)

      img_batch = x_train[np.random.randint(0,x_train.shape[0], size=batch_size)]
      imgs = np.concatenate([fake_imgs, img_batch])

      labels = np.asarray([0.95]*batch_size + [0.05]*batch_size).astype('float32')

      # train discriminator
      ramen_discriminator.trainable = True
      Disc_trained = ramen_discriminator.fit(imgs,labels, verbose=0)

      # train GAN
      y_generator = np.asarray([.05]*batch_size).astype('float32')
      ramen_discriminator.trainable = False
      GAN_trained = ramenGAN.fit(noise_input, y_generator, verbose=0)

      input_g = np.random.rand(1,100)
      preds = (ramen_generator.predict(input_g) + 1.0) / 2.0

      if epoch%10 == 0:
        show_image(preds[0])
        for pred in range(preds.shape[0]):
          fig = px.imshow(preds[pred])
          if epoch%20 == 0:
            fig.write_image(f"gan_output/epoch{epoch}.jpg")
            

In [ ]:
train(800, 30)